# Projet Algorithmique et programmation ISE Eco

* Installation de bibliothèque nécessaire

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time


# THEME 1: Les données économique de la Côte d'Ivoire

* données issue de l'Université de Sherbrooke, là nous allons mettre en place un code qui va scraper les données économiques sur plusieurs années. pour cela nous definissons une fonction qu prend en paramètre le code du pays. Evidement le site est dynamique;nous avons recours à Selenium pour parcouru 

In [2]:


# Liste des indicateurs et leurs codes
indicateurs = {
    'idh': 'SP.POP.IDH.IN',
    'iphc': 'FP.CPI.TOTL',
    'gini': 'PMQUANDL.GINI.V1',
    'pib_courant': 'NY.GDP.MKTP.KD',
    'pib_hbt': 'NY.GDP.PCAP.KD',
    'va_agri': 'NV.AGR.TOTL.ZS',
    'va_indus': 'NV.IND.TOTL.ZS',
    'va_serv': 'NV.SRV.TOTL.ZS',
}

# Modèle d'URL
url_template = (
    "https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?"
    "codeTheme=2&codeStat={stat_code}&codePays={country_code}&optionsPeriodes=Aucune"
)

# Fonction pour générer les URLs
def generate_urls(indicateurs, country_code):
    return {nom: url_template.format(stat_code=code, country_code=country_code) for nom, code in indicateurs.items()}

# Fonction principale de scraping
def scrap_data(country_code):
    urls_with_filenames = generate_urls(indicateurs, country_code)

    for nom_fichier, url in urls_with_filenames.items():
        driver = None
        try:
            # Initialiser le navigateur Edge
            driver = webdriver.Edge()

            # Ouvrir l'URL dans le navigateur
            driver.get(url)

            # Attendre que la page se charge complètement
            wait = WebDriverWait(driver, 20)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody#listeDonnee td")))

            # Trouver les éléments <td> contenant les années et les valeurs
            data_elements = driver.find_elements(By.CSS_SELECTOR, "tbody#listeDonnee td")

            # Parcourir les éléments et extraire le texte
            data = [elem.text for elem in data_elements]

            # Extraire les années et les valeurs en utilisant le slicing pour les séparer
            annees = data[::3]
            croissance_pop = data[1::3]

            # Générer le nom complet du fichier en ajoutant l'extension CSV
            nom_fichier_csv = f'{nom_fichier}.csv'

            # Enregistrer les données dans un fichier CSV
            with open(nom_fichier_csv, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Année', nom_fichier])  # Écrire l'en-tête
                for annee, taux_croissance_pop in zip(annees, croissance_pop):
                    writer.writerow([annee, taux_croissance_pop])

            print(f"Les données de l'URL {url} ont été enregistrées dans {nom_fichier_csv} avec succès.")

        except Exception as e:
            print(f"Une erreur s'est produite en traitant {url}: {e}")

        finally:
            # Fermer le navigateur proprement
            if driver:
                driver.quit()

# Exemple d'utilisation pour la Côte d'Ivoire
scrap_data("CIV")  # Code ISO du pays Côte d'Ivoire


Les données de l'URL https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?codeTheme=2&codeStat=SP.POP.IDH.IN&codePays=CIV&optionsPeriodes=Aucune ont été enregistrées dans idh.csv avec succès.
Les données de l'URL https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?codeTheme=2&codeStat=FP.CPI.TOTL&codePays=CIV&optionsPeriodes=Aucune ont été enregistrées dans iphc.csv avec succès.
Les données de l'URL https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?codeTheme=2&codeStat=PMQUANDL.GINI.V1&codePays=CIV&optionsPeriodes=Aucune ont été enregistrées dans gini.csv avec succès.
Les données de l'URL https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?codeTheme=2&codeStat=NY.GDP.MKTP.KD&codePays=CIV&optionsPeriodes=Aucune ont été enregistrées dans pib_courant.csv avec succès.
Les données de l'URL https://perspective.usherbrooke.ca/bilan/servlet/BMTendanceStatPays?codeTheme=2&codeStat=NY.GDP.PCAP.KD&codePays=CIV&optionsPeriodes=Aucune

* ANSTAT : l'Agence Nationale de la Statistique de Cote d'Ivoire, nous avons scraper les données demographque, économique et portant sur les conditions de vie. le site etant dynamique, nous avons utilisé Selenium pour parcouru les differentes categores citées haut.

In [5]:

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

driver.get("https://www.anstat.ci")

wait = WebDriverWait(driver, 10)

# Définir le chemin du dossier "données" de manière explicite
data_folder = os.path.join(os.getcwd(), 'données')

# Créer le dossier "données" s'il n'existe pas
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Fonction pour scraper les données d'un menu donné
def scrape_data(menu_id, group_name):
    # Cliquer sur l'onglet correspondant
    menu = wait.until(EC.element_to_be_clickable((By.ID, menu_id)))
    menu.click()

    # Pause pour laisser le contenu se charger (ajuster selon les besoins)
    time.sleep(2)

    # Attendre que les éléments soient visibles
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item")))

    # Scraper les données
    items = driver.find_elements(By.CLASS_NAME, "item")
    group_data = []

    for item in items:
        try:
            # Extraire les données
            title = item.find_element(By.CLASS_NAME, "titleIndicateur").text.strip()
            year = item.find_element(By.TAG_NAME, "small").text.strip()
            value_element = item.find_element(By.CLASS_NAME, "valIndicateur")
            value = value_element.text.split("\n")[0].strip()
            unit = value_element.find_element(By.TAG_NAME, "sup").text.strip()
            update_date = item.find_elements(By.TAG_NAME, "p")[2].text.split(": ")[1]
            coverage = item.find_elements(By.TAG_NAME, "p")[3].text.split(": ")[1]
            description = item.find_element(By.CLASS_NAME, "ind_descript").text.strip()
            # Ajouter aux données
            group_data.append({
                "title": title,
                "year": year,
                "value": value + " " + unit,
                "update_date": update_date,
                "coverage": coverage,
                "description": description
            })
        except Exception as e:
            print(f"Erreur pour un item : {e}")

    # Sauvegarder les données dans un fichier CSV dans le dossier "données"
    file_path = os.path.join(data_folder, f'{group_name}_data.csv')
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "year", "value", "update_date", "coverage", "description"])
        writer.writeheader()
        writer.writerows(group_data)

    print(f"Données extraites pour {group_name} : {len(group_data)} items. Fichier enregistré dans {file_path}")

# Scraper les données pour chaque menu
scrape_data("fdv", "demographic")  # Démographie
scrape_data("midv", "economic")    # Économie
scrape_data("enddv", "living")     # Conditions de vie

# Fermer le navigateur
driver.quit()


Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Données extraites pour demographic : 4 items. Fichier enregistré dans d:\ISE\COURS\ALGO_PRO\Projet_Algo_DROH_Romaric\Scripts\données\demographic_data.csv
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un item : list index out of range
Erreur pour un ite

# THEME 2: EMPLOI

Par la suite nous allons scraper les offres d'emploi sur google carrière, nous avons voulu utlser BaeutifoulSoup et requests comme outils de scraping. Pour cela, nous avons énumerer les urls, faire appel à des requêtes pour parcouru les pages. car dans un domane il y'a plusieurs offres reparties sur differentes pages. Une fois nous avons voulu nous passer de Selenium


In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re


urls = [
    'https://www.google.com/about/careers/applications/jobs/results/?hl=fr-CA',
    'https://www.google.com/about/careers/applications/jobs/results/?tag=ai-spotlight&hl=en_US',
    'https://www.google.com/about/careers/applications/jobs/results/?company=Google&company=YouTube&jlo=en_US&q=Hardware%20Engineer&hl=en_US',
    'https://www.google.com/about/careers/applications/jobs/results/?distance=50&hl=en_US&jlo=en_US&q=%22Platforms%20and%20Ecosystems%22&hl=en_US',
    'https://www.google.com/about/careers/applications/jobs/results/?distance=50&hl=en_US&jlo=en_US&q=%22Site%20reliability%20engineer%22&hl=en_US'
]


output_file = 'offres_emploi_multiple.csv'

# En-têtes pour le CSV
headers = ['Domaine', 'Titre du Poste', 'Nom de l\'Entreprise', 'Lieu', 'Compétences']

# Fonction pour scraper et enregistrer les données
def scraper_to_csv(url, output_file):
    try:
        page_number = 1  
        while True:
      
            page_url = f"{url}&start={(page_number - 1) * 20}"
            
        
            response = requests.get(page_url)
            response.raise_for_status()  
    
            soup = BeautifulSoup(response.text, 'html.parser')

            job_titles = soup.find_all('h3', class_='QJPWVe')  
            companies = soup.find_all('span', class_='RP7SMd')
            locations = soup.find_all('span', class_='pwO9Dc vo5qdf')
            skills = soup.find_all('div', class_='Xsxa1e')

           
            pagination_status = soup.find('div', class_='VfPpkd-wZVHld-gruSEe-j4LONd')
            if pagination_status:
                rows_text = pagination_status.text.strip()
               
                match = re.search(r'(\d+)\s*(sur|of)\s*(\d+)', rows_text)
                if match:
                    total_rows = int(match.group(3))  
                else:
                    print(f"Le format de pagination est inattendu : {rows_text}")
                    total_rows = 0  
            with open(output_file, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if page_number == 1:  
                    writer.writerow(headers) 

                
                for title, company, location, skill in zip(job_titles, companies, locations, skills):
                   
                    title_text = title.text.strip() if title else "N/A"

                   
                    company_name = company.find('span').text.strip() if company.find('span') else "N/A"

                    
                    location_text = " / ".join([loc.text.strip() for loc in location.find_all('span') if loc.text.strip()]) if location else "N/A"

              
                    skill_list = skill.find_all('li')
                    skills_text = " / ".join([skill_item.text.strip() for skill_item in skill_list]) if skill_list else "N/A"

                    # Écrire la ligne dans le CSV
                    writer.writerow([url, title_text, company_name, location_text, skills_text])

            # Si nous avons parcouru toutes les pages, sortir de la boucle
            if page_number * 20 >= total_rows:
                break

            
            page_number += 1
            print(f"Page {page_number} traitée pour l'URL: {url}...")

           
            time.sleep(2)

        print(f"Les données ont été enregistrées dans {output_file}")

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de la page : {e}")


for url in urls:
    print(f"Scraping de l'URL : {url}")
    scraper_to_csv(url, output_file)


Scraping de l'URL : https://www.google.com/about/careers/applications/jobs/results/?hl=fr-CA
Les données ont été enregistrées dans offres_emploi_multiple.csv
Scraping de l'URL : https://www.google.com/about/careers/applications/jobs/results/?tag=ai-spotlight&hl=en_US
Les données ont été enregistrées dans offres_emploi_multiple.csv
Scraping de l'URL : https://www.google.com/about/careers/applications/jobs/results/?company=Google&company=YouTube&jlo=en_US&q=Hardware%20Engineer&hl=en_US
Page 2 traitée pour l'URL: https://www.google.com/about/careers/applications/jobs/results/?company=Google&company=YouTube&jlo=en_US&q=Hardware%20Engineer&hl=en_US...
Page 3 traitée pour l'URL: https://www.google.com/about/careers/applications/jobs/results/?company=Google&company=YouTube&jlo=en_US&q=Hardware%20Engineer&hl=en_US...
Page 4 traitée pour l'URL: https://www.google.com/about/careers/applications/jobs/results/?company=Google&company=YouTube&jlo=en_US&q=Hardware%20Engineer&hl=en_US...
Page 5 traité